# Get necessary imports

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import faiss
from sentence_transformers import SentenceTransformer

In [11]:
data = []
print(f"Loading data from 'recipes_for_embeddings.jsonl'...")
assert os.path.exists('recipes_for_embeddings.jsonl'), "Data file not found!"
with open('recipes_for_embeddings.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
metadata = [d for d in data]  # Keep full object to pass to Qwen later
print(f"Loaded {len(data)} documents.")

Loading data from 'recipes_for_embeddings.jsonl'...
Loaded 18222 documents.


In [12]:
# Load embedding model (must match the one used in build_index.ipynb)
print("Loading embedding model...")
embed_model = SentenceTransformer('BAAI/bge-m3')

# Load FAISS index
print("Loading FAISS index...")
index = faiss.read_index('recipe_index.faiss')
print(f"Index loaded with {index.ntotal} vectors.")

Loading embedding model...
Loading FAISS index...
Index loaded with 18222 vectors.


In [14]:
# --- RETRIEVAL FUNCTION ---
def search(query, k=3):
    # 1. Embed the query
    query_vec = embed_model.encode([query], convert_to_tensor=False)
    
    # 2. Search FAISS
    _, indices = index.search(query_vec, k)
    
    # 3. Retrieve actual documents
    results = []
    for idx in indices[0]:
        results.append(metadata[idx])
    return results

# Instantiating model and tokenizer

In [15]:
model_id = "Qwen/Qwen2.5-14B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [ ]:
def generate_rag_response(user_query):
    # 1. Retrieve Context
    retrieved_docs = search(user_query, k=3)
    
    # 2. Format Context for the Prompt
    context_str = ""
    for i, doc in enumerate(retrieved_docs):
        context_str += f"Recipe {i+1}: {doc['title']}\nContent: {doc['text_for_embedding']}\n\n"

    # 3. Construct Qwen Chat Template
    # Qwen performs best with a clear distinction between system instructions and context
    messages = [
        {"role": "system", "content":
         "You are a helpful, health-conscious AI cooking assistant. "
         "IMPORTANT: Only use information from the provided recipe context. "
         "Always cite which recipe(s) you're referencing by name. "
         "If the answer is not in the context, explicitly say 'I don't have this information in the available recipes.' "
         "Do not make up recipes or ingredients that aren't in the context."},
        {"role": "user", "content": f"Context:\n{context_str}\n\nQuestion: {user_query}"}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 4. Display retrieved recipes for manual validation
    print("=== RETRIEVED RECIPES ===")
    for i, doc in enumerate(retrieved_docs):
        print(f"{i+1}. {doc['title']}")
    print("=========================\n")
    
    # 5. Generate (Mock call if model not loaded)
    print("--- INPUT PROMPT ---")
    print(text)
    print("--------------------")
    
    return retrieved_docs  # Return for validation

In [ ]:
def validate_response(llm_response, retrieved_docs):
    """
    Check if the LLM response uses information from retrieved documents
    """
    # Extract recipe titles from retrieved docs
    retrieved_titles = [doc['title'].lower() for doc in retrieved_docs]
    
    # Extract all ingredients from retrieved docs
    all_ingredients = set()
    for doc in retrieved_docs:
        # Parse ingredients from text_for_embedding
        text = doc['text_for_embedding'].lower()
        if 'ingredients:' in text:
            ing_section = text.split('ingredients:')[1].split('recipe:')[0]
            all_ingredients.update(ing_section.split(','))
    
    all_ingredients = {ing.strip() for ing in all_ingredients if ing.strip()}
    
    print("\n=== VALIDATION ===")
    print(f"Retrieved Recipes: {retrieved_titles}")
    print(f"Available Ingredients: {list(all_ingredients)[:10]}...")  # Show first 10
    
    # Check if response mentions retrieved recipes
    llm_lower = llm_response.lower()
    mentioned_recipes = [title for title in retrieved_titles if title in llm_lower]
    
    if mentioned_recipes:
        print(f"✓ Response cites: {mentioned_recipes}")
    else:
        print("⚠ WARNING: Response doesn't cite any retrieved recipes!")
    
    print("==================\n")
    
    return {
        'retrieved_titles': retrieved_titles,
        'mentioned_recipes': mentioned_recipes,
        'likely_grounded': len(mentioned_recipes) > 0
    }

In [18]:
query = "I have eggplants and garlic, what can I make?"
generate_rag_response(query)

--- INPUT PROMPT ---
<|im_start|>system
You are a helpful, health-conscious AI cooking assistant.You specialize in generating, adapting, and evaluating recipes based on user dietary needs and available ingredients.Think step-by-step and provide clear, friendly responses<|im_end|>
<|im_start|>user
Context:
Recipe 1: Roasted Eggplant and Garlic Dip 
Content: Ingredients: eggplant, extra-virgin olive oil, red-wine vinegar or to taste, small heads garlic. Recipe: Roasted Eggplant and Garlic Dip

Recipe 2: Herbed Eggplant with Tomatoes, Onion and Garlic 
Content: Ingredients: canned diced tomatoes in juice drained, chopped fresh parsley, chopped white onion, dried oregano, extra-virgin olive oil, fresh lemon juice, garlic cloves minced, medium eggplants, pita bread cut into wedges, plain yogurt, red wine vinegar. Recipe: Herbed Eggplant with Tomatoes, Onion and Garlic

Recipe 3: Garlicky Eggplant Salad 
Content: Ingredients: chopped fresh oregano, chopped fresh oregano or dried crumbled, fr